In [1]:
import numpy as np
from PIL import Image
from pickle import load
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
from utils.model import CNNModel, generate_caption_beam_search
import os
from utils.timer import Timer
from config import config
import json

Using TensorFlow backend.
/Users/kevinhou/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
"""
    *Some simple checking
"""
assert type(config['max_length']) is int, 'Please provide an integer value for `max_length` parameter in config.py file'
assert type(config['beam_search_k']) is int, 'Please provide an integer value for `beam_search_k` parameter in config.py file'

# Extract features from each image in the directory
def extract_features(filename, model, model_type):
    if model_type == 'inceptionv3':
        from keras.applications.inception_v3 import preprocess_input
        target_size = (299, 299)
    elif model_type == 'vgg16':
        from keras.applications.vgg16 import preprocess_input
        target_size = (224, 224)
    # Loading and resizing image
    image = load_img(filename, target_size=target_size)
    # Convert the image pixels to a numpy array
    image = img_to_array(image)
    # Reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # Prepare the image for the CNN Model model
    image = preprocess_input(image)
    # Pass image into model to get encoded features
    features = model.predict(image, verbose=0)
    return features

In [16]:
timer = Timer()

In [4]:
# Load the tokenizer
tokenizer_path = config['tokenizer_path']
tokenizer = load(open(tokenizer_path, 'rb'))

# Max sequence length (from training)
max_length = config['max_length']

In [5]:
models = [
    {
        'name': 'VGG16',
        'load_path': 'model_data/model_vgg16_epoch-11_train_loss-2.2639_val_loss-3.1549.hdf5',
        'model_type': 'vgg16'
    },
    {
        'name': 'InceptionV3 Beam 3',
        'load_path': 'model_data/model_inceptionv3_epoch-20_train_loss-2.3803_val_loss-2.8221-beam3.hdf5',
        'model_type': 'inceptionv3'
    },
    {
        'name': 'InceptionV3 Beam 5',
        'load_path': 'model_data/model_inceptionv3_epoch-19_train_loss-2.3802_val_loss-2.8636-beam5.hdf5',
        'model_type': 'inceptionv3'
    }
]

# Load Models

In [9]:
timer.start('Model Load')

image_models = {}

for i, model in enumerate(models):
    model_name = model['name']
    model_type = model['model_type']
    
    timer.start(model_name)
    
    # Load the caption model and image model
    caption_model = load_model(model['load_path'])
    
    if model_type not in image_models:
        image_models[model_type] = CNNModel(model_type)
    
    # Set properties back on model object
    models[i]['model'] = caption_model
    
    print("\t%s: %d seconds" % (model_name, timer.end('Model Load')))

print("Total Model Load Time: %d seconds" % timer.end('Model Load'))

	VGG16: 9 seconds

	InceptionV3 Beam 3: 31 seconds
	InceptionV3 Beam 5: 43 seconds
Total Model Load Time: 43 seconds


# Test on Image

In [26]:
image_path = config['test_data_path'] + 'people-working.jpg'

In [27]:
timer.start('Image Analysis')
print("[START]")

for i, model in enumerate(models):
    model_name = model['name']
    model_type = model['model_type']
    image_model = image_models[model_type]
    caption_model = model['model']
    
    timer.start(model_name)
    
    image_features = extract_features(image_path, image_model, model_type)
    generated_caption_seq = generate_caption_beam_search(caption_model, tokenizer, image_features, max_length, beam_index=config['beam_search_k'])
    generated_caption = ' '.join(generated_caption_seq.split()[2:len(generated_caption_seq.split())-1])
    
    models[i]['caption'] = generated_caption
    
    print("\n%s" % model_name)
    print("\tCaption: %s" % generated_caption)
    print("\tDuration: %d seconds" % timer.end(model_name))

print("\n[END] Image Analysis Time: %d" % timer.end('Image Analysis'))

[START]

VGG16
	Caption: woman is sitting at a table at a restaurant
	Duration: 2 seconds

InceptionV3 Beam 3
	Caption: group of people sit in front of a restaurant
	Duration: 4 seconds

InceptionV3 Beam 5
	Caption: group of people are standing in front of a glass fountain
	Duration: 4 seconds

[END] Image Analysis Time: 10


In [14]:
results = [x['caption'] for x in models]
with open('results.json', 'w') as f:
    json.dump(results, f)